# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

PEFT Technique:
I selected LoRA (Low Rank Adaptation) for its efficiency in fine-tuning large language models with fewer parameters. This approach helps reduce computational costs while maintaining performance by introducing low-rank adaptations to the model's attention mechanism, thereby decreasing computational complexity and memory usage.

Model:
For sentiment analysis, I chose the GPT-2ForSequenceClassification architecture. GPT-2 is renowned for its effectiveness across various NLP tasks, including classification. By using GPT-2ForSequenceClassification, we leverage the pre-trained GPT-2 weights, which capture rich linguistic patterns and contexts, potentially enhancing performance.

Evaluation Approach:
I evaluate the model both before and after fine-tuning using the Trainer's evaluate() method. This direct comparison ensures the effectiveness of the fine-tuning process. By using the same metrics and procedures on the validation dataset, we can objectively assess the impact of fine-tuning on model performance.

Fine-Tuning Dataset:
For fine-tuning, we utilize the Rotten Tomatoes, which is well-suited for the sentiment analysis task at hand.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install datasets #for colab

In [2]:
!pip list | grep "peft" || pip install peft

peft                             0.12.0


In [3]:
# importing dependencies.

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import pandas as pd

In [4]:
# Loading dataset (train and test splits)
splits = ["train", "test"]
dataset = {split: load_dataset("cornell-movie-review-data/rotten_tomatoes", split=split) for split in splits}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
dataset

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 8530
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 1066
 })}

In [6]:
def compute_stats(dataset, subset_name):
    """
    Compute and print statistics for a subset of a HuggingFace dataset.

    :param dataset: A Hugging Face dataset object.
    :param subset_name: A string with the name of the subset.

    :returns: None
    """
    subset = dataset[subset_name]
    sentences = subset['text']
    labels = subset['label']

    # Number of samples in subset
    num_samples = subset.num_rows
    print(f'Number of samples in {subset_name} subset: {num_samples}')

    # Maximum and minimum length of sequences in the subset
    sentence_lengths = [len(sentence) for sentence in sentences]
    max_length = max(sentence_lengths)
    min_length = min(sentence_lengths)
    print(f'Max length of sentence in {subset_name} subset: {max_length}')
    print(f'Min length of sentence in {subset_name} subset: {min_length}')

    # Labels in the subset
    unique_labels = set(labels)
    print(f'Labels in {subset_name}: {unique_labels}')

    # Percentages of each label in the subset
    label_counts = {label: labels.count(label) for label in unique_labels}
    label_percentages = {label: (count / num_samples) * 100 for label, count in label_counts.items()}

    print(f'Percentages for each label in {subset_name} subset:')
    for label, percentage in label_percentages.items():
        print(f'- Label {label}: {round(percentage, 2)}%')


In [7]:
compute_stats(dataset, "train")

Number of samples in train subset: 8530
Max length of sentence in train subset: 267
Min length of sentence in train subset: 4
Labels in train: {0, 1}
Percentages for each label in train subset:
- Label 0: 50.0%
- Label 1: 50.0%


In [8]:
compute_stats(dataset, "test")

Number of samples in test subset: 1066
Max length of sentence in test subset: 261
Min length of sentence in test subset: 14
Labels in test: {0, 1}
Percentages for each label in test subset:
- Label 0: 50.0%
- Label 1: 50.0%


# Load Tokenizer and tokenize the dataset

In [9]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
def preprocess(examples): return tokenizer(examples["text"], padding=True, truncation =True)

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(preprocess, batched = True)

In [12]:
base_model = AutoModelForSequenceClassification.from_pretrained('gpt2',
                                                      num_labels=2,
                                                      id2label={0: "NEGATIVE", 1: "POSITIVE"},
                                                      label2id={"NEGATIVE": 0, "POSITIVE": 1})

base_model.config.pad_token_id = tokenizer.pad_token_id

for param in base_model.base_model.parameters():
    param.requires_grad = False

base_model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [13]:
base_model.score

Linear(in_features=768, out_features=2, bias=False)

In [14]:
def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [15]:
training_args = TrainingArguments(
    output_dir="./model_output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

pretrain_trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

base_model_evaluation = pretrain_trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
print("Evaluation result before fine-tuning:")
base_model_evaluation

Evaluation result before fine-tuning:


{'eval_loss': 4.876796722412109,
 'eval_accuracy': 0.49906191369606,
 'eval_runtime': 284.3872,
 'eval_samples_per_second': 3.748,
 'eval_steps_per_second': 0.236}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [17]:
from peft import  get_peft_model, LoraConfig, TaskType

In [18]:
model = AutoModelForSequenceClassification.from_pretrained('gpt2',
                                                      num_labels=2,
                                                      id2label={0: "NEGATIVE", 1: "POSITIVE"},
                                                      label2id={"NEGATIVE": 0, "POSITIVE": 1})

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.config.pad_token_id = tokenizer.pad_token_id

In [20]:
config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=['c_attn', 'c_proj'],
                    lora_dropout=0.1,
                    bias="none",
                    task_type=TaskType.SEQ_CLS
                )

peft_model = get_peft_model(model, config)
peft_model.print_trainable_parameters()

trainable params: 812,544 || all params: 125,253,888 || trainable%: 0.6487


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [21]:
# Rename 'label' column to 'labels'
tokenized_dataset["train"] = tokenized_dataset["train"].map(lambda e: {'labels': e['label']}, batched=True, remove_columns=['label'])
tokenized_dataset["test"] = tokenized_dataset["test"].map(lambda e: {'labels': e['label']}, batched=True, remove_columns=['label'])

# Print to verify
print(tokenized_dataset["train"][0]["text"])
print(tokenized_dataset["train"][0]["labels"])

# Set the format for PyTorch
tokenized_dataset["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_dataset["test"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
1


In [22]:
trainer_peft = Trainer(
model = peft_model,
args = TrainingArguments(
    output_dir = "./lora_model_output",
    learning_rate = 2e-5,
    logging_strategy = "steps",
    per_device_train_batch_size = 12,
    per_device_eval_batch_size = 12,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True
),
train_dataset = tokenized_dataset["train"],
eval_dataset = tokenized_dataset["test"],
tokenizer = tokenizer,
data_collator =  DataCollatorWithPadding(tokenizer=tokenizer),
compute_metrics = compute_metrics,
)



In [23]:
trainer_peft.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.672400,0.622547,0.699812


TrainOutput(global_step=711, training_loss=0.6619596843478046, metrics={'train_runtime': 5535.2051, 'train_samples_per_second': 1.541, 'train_steps_per_second': 0.128, 'total_flos': 330172861962240.0, 'train_loss': 0.6619596843478046, 'epoch': 1.0})

In [24]:
peft_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [25]:
import torch
from peft import AutoPeftModelForSequenceClassification

NUM_LABELS = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora", num_labels=NUM_LABELS, ignore_mismatched_sizes=True).to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
lora_model.config.pad_token_id = tokenizer.pad_token_id

In [27]:
training_args = TrainingArguments(
    output_dir="./data/sentiment_analysis",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

finetuned_trainer = Trainer(
    model=lora_model,  # The fine-tuned PEFT model.
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Evaluate the fine-tuned model on the validation set
finetuned_results = finetuned_trainer.evaluate()

# Print the evaluation results for the fine-tuned model
print("Evaluation results for the fine-tuned model:")
finetuned_results

Evaluation results for the fine-tuned model:


{'eval_loss': 0.622546911239624,
 'eval_accuracy': 0.699812382739212,
 'eval_runtime': 253.769,
 'eval_samples_per_second': 4.201,
 'eval_steps_per_second': 0.264}

# Conclusion

The evaluation results highlight significant improvements in the performance of the PEFT (Parameter-Efficient Fine-Tuned) model compared to the base model, demonstrating the effectiveness of the fine-tuning process. Using LoRA (Low Rank Adaptation) for its efficiency in fine-tuning, we aimed to reduce computational costs while maintaining performance by introducing low-rank adaptations to the model's attention mechanism.

For sentiment analysis, the GPT-2ForSequenceClassification architecture was chosen due to its renowned effectiveness across various NLP tasks. By leveraging pre-trained GPT-2 weights, we harness rich linguistic patterns and contexts, enhancing the model's performance.

Before fine-tuning, the evaluation results indicated an eval_loss of 4.8768, an eval_accuracy of 0.4991, an eval_runtime of 284.3872 seconds, eval_samples_per_second of 3.748, and eval_steps_per_second of 0.236. After fine-tuning on the Rotten Tomatoes dataset, the model showed a substantial reduction in eval_loss to 0.6225 and an increase in eval_accuracy to 0.6998. Additionally, the fine-tuned model demonstrated improved evaluation runtimes (253.769 seconds vs. 284.3872 seconds) with higher samples per second (4.201 vs. 3.748) and steps per second (0.264 vs. 0.236).

These improvements underscore the impact of parameter-efficient fine-tuning in optimizing the model for sentiment analysis. The results validate that fine-tuning with LoRA significantly enhances loss reduction and predictive accuracy while being computationally efficient, making it a highly effective approach for fine-tuning large language models like GPT-2ForSequenceClassification.